# Network Visualization
A place to view networks while experimenting with construction in JSON files and/or *NetworkBuilder.py*.

In [1]:
import sys
sys.path.insert(0, "../python")
from helper import create_network
import tensorflow as tf
%load_ext autoreload
%autoreload 2

In [12]:
# Credit: https://stackoverflow.com/questions/38189119/simple-way-to-visualize-a-tensorflow-graph-in-jupyter/38192374#38192374
from IPython.display import clear_output, Image, display, HTML
import numpy as np

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [13]:
phi = 4
channels= 3
num_actions = 4
learning_rate = 0.00025
params_file_path = None
output_directory = "./network_visualization/results"
scope = ""

## Batch normalization

In [17]:
tf.reset_default_graph()
sess = tf.Session()
net_file = "./network_visualization/batch_norm.json"
network = create_network(net_file,
                         phi=phi, 
                         num_channels=channels, 
                         num_actions=num_actions,
                         learning_rate=learning_rate,
                         params_file=params_file_path,
                         output_directory=output_directory,
                         session=sess,
                         scope=scope)
show_graph(tf.get_default_graph().as_graph_def())